# Slot-value Definitions & Generation Rules

This document is mainly based on the code. I've also added the counterpart definitions in DSTC and our ontology.

## State Definition
      state  = {"role": "user",
                "what": ("null", "null"), // title
                "when_start": ("null", "null"), // startTime
                "duration": ("null", "null"),
                "who": ("null", "null"), // invitee
                "where": ("null", "null"), // location
                "day": ("null", "null")} // startDay
                
- The 1st element in the pair is one of {null, weak, know}, meaning that the user has not yet mentioned the slot, the sys has low confidence of the slot, the sys has high confidence of the slot.

- The 2nd element in the pair is the value of the slot.
  role: user/sys, means that it is the user/sys's turn to speak
  what: a string
  when_start: (start_hour, start_minute)
  duration: (duration_hour, duration_minute)
  who: a list of names
  where: a string
  day: a string (of the form "Month/Day", or "this/next Monday/..."
  
## Action Definitions
- User actions

      user_inform(slot, value) // inform
        e.g. user_inform(time, 13:00) - "Set the time to 1 pm."
          
      user_affirm // affirm
        e.g. user_affirm - "Correct."
        
      user_ack // ack
        e.g. user_ack - "Yes/OK"
        
      user_finish // bye
        e.g. user_finish - "That's all/Good bye."
        
      user_start // hello
        e.g. user_start - "Hi, I want to set an reminder."
        
      user_restart // restart
        e.g. user_restart - "Well, let's start over again."
        
      user_report // similar to request. in DSTC request is for a slot, but here we ask a full report
        e.g. user_report - "Tell me everything you heard."
        
      user_dont_want_report // similar to negate. However, negate can only generate 'No/Wrong'
                            // but this action generate more natural response like
                            // 'don't report me', 'no need for the report'

- System actions

      sys_request(slot) // request
        e.g. sys_request(who) - "Who do you want to go with?"
        
      sys_ack // similar to sys_affirm. However, it is strange that DSTC never use sys_ack
              // ack - 'OK/I hear you', affirm - 'you are right'. I think ack is a better name.
        e.g. sys_ack - "OK."
        
      sys_expl_confirm (slot) // expl_confirm
                              // the value is obtained from the state
        e.g. sys_expl_confirm(when) - "Are you sure that it starts at 5 pm?"
        
      sys_finish // bye
        e.g. sys_finish - "The reminder has been set."
        
      sys_report
        e.g. sys_report - "You said you want to go to KFC 1 pm tomorrow with Alice and Bob."
        
      sys_report_request
        e.g.. sys_report_request - "Do you want to review the reminder?"
        
- Generation Rules

        \rng means the branch is chosen randomly. Sometimes the prefix (sys_, user_) is omitted to make the representation more succinct.
        The states are conceptually divided into two groups: role=user and role=sys.
        role=user means the user will speak (including hi, inform, etc).
        role=sys means some slot is weak, so the system needs to make confirmation.
        
    - role=sys:
    
            -> some slot is weak -> expl_confirm slots -> affirm\rng -> role=sys
                                                       -> user change weak slots\rng -> role=sys
            -> all slots are know -> report_request -> ack\rng -> report -> affirm\rng -> finish
                                                                         -> role=user\rng (to user state so some slots may change)
                                                    -> dont_want_report\rng -> finish
            -> some slots are null -> request null slots -> role=user
            
    - role=user:
    
            -> start/rng (only when not yet started) -> ack -> role=user
            -> restart/rng -> ack -> role=sys
            -> user_report/rng -> sys_report -> role=user
            -> finish/rng -> role=sys
            -> inform/rng (prob of inform null/know slot is different) -> role=sys
            
- Transition Probabilities

        The code will uniform the probabilities, so don't worry sum(p(x)) != 1.
        
         conf["user_start_prob"] = 0.3
             If true, the user starts dialog with hello message ('hi, i want to set a reminder'). Otherwise starts the dialog immediately.
             
         conf["user_ack_expl_confirm_prob"] = 0.9
             If true, the user say 'yes' after the sys asks for confirmation.
             
         conf["user_ack_report_request_prob"] = 0.9
             Prob that the user wants a report.
             
         conf["user_restart_prob"] = 0.05
             Prob of restarting the system.
             
         conf["user_report_prob"] = 0.05
             Prob of requesting a report (in the middle of the dialog)
             
         conf["user_finish_prob"] = 0.05
             Prob that the user says he is finished (while some fields are still null).
             
         conf["user_inform_prob"] = 0.85
             Prob that the user informs a slot.
             
         conf["user_inform_skip_prob"] = 0.4
             Prob that the user skips a slot.
             
         conf["user_inform_new_weak_prob"] = 0.1
             Prob that the new informed slot is weak.
             
         conf["user_inform_new_know_prob"] = 0.9
             Prob that the new informed slot is know.

         conf["user_inform_change_prob"] = 0.05
             Prob that the user changes the value of a slot.
             
         conf["report_correct_prob"] = 0.9
             Prob that the report is totally correct.
        
- (Incomplete) Flow Graph

        My first thought was to encoder all the rules as a finite state transducer (similar to a automata). But I thought it is too time consuming to draw all the details, while most of things are almost the same. So the following is only an incomplete figure of my previous effort.
![Alt text](./fig.jpg)